# COMP90042 Assignment #2: Cross-language Information Retreival

```Student Name: Yun Wang
Student ID: 672323```

Please do not edit this field. It must be left as is for use in marking.

## General info

**Due date**: 5pm, Wed 25th May

**Submission method**: see LMS

**Submission materials**: completed copy of this ipython notebook. *Do not upload the corpora*, you can assume these files will be extracted into the same folder as the code.  *Do not use absolute paths* such as paths to files on your machine, use *relative paths* instead. *Include output* in the ipython notebook, and ensure your code can run sucessfully from scratch (e.g., use *Kernel -> Restart and Run All* menu option to test.)

**Late submissions**: -10% per day, no late submissions after the first week

**Marks**: 25% of mark for class

**Overview**: For this project, you'll be building a cross language information retreival system (CLIR), which is capable of searching for important text documents written in a English given a query in German, and displaying the results in German. This incorporates machine translation, information retrieval using a vector space model, and IR evaluation. A key focus of this project is critical analysis and experimental evaluation, for which you will need to report on the relative merits of various options. 

**Materials**: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages as used for workshops and the previous assignments. The main part of this assignment should be done entirely in python, using built-in python packages and the packages used in the class (NLTK, Numpy, Scipy, Matplotlib, Sci-kit Learn, and Gemsim). Please do no use any other 3rd party packages. Note that some choices of the *extended part* of the assignment may require you to work with other tools, such as 3rd party translation tools or retreival engines, in which case this part of the assignment will require working in another environment, although you should report your results in the ipython notebook and attach your other files (code, shell scripts etc) with your submission.   

You are encouraged to make use of the iPython notebooks released for this class as well as other online documentation to guide your responses, but you should not copy directly from any source. Adapting code from the class notebooks is permitted.

**Data**: The other data you will need are:
  - parallel German-English corpus.  This data comes from the Europarl corpus, which is a collection of debates held in the EU parliament over many years, translated into several EU languages. This data was collated as part of the annual WMT translation competitions. The data has been tokenised, sentence aligned using the Church-Gale algorithm. 
  - CLIR evaluation corpus, comprising a large collection of English documents sourced from Wikipedia along with several German query strings and relevance judgements for each document and query pair. 
  
The data files are as follows:
  - *bitext-small.{en,de}* is the sentence-aligned parallel corpus, of a small enough size for you to use for developing word-alignment tools. You may wish to work on a smaller subset during code development.
  - *bitext-large.{en,de}* is a much larger version of the above corpus, for language modelling and translation.
  - *newstest2013.{en,de}* is a separate small parallel corpus reserved for evaluation.
  - *dev.{docs,queries,qrel}* is a set of documents in English, queries in German and query relevance judgements, for the development of the IR components and evaluation.
  
For details of the datasets, please see:
> Philipp Koehn. *Europarl: A Parallel Corpus for Statistical Machine Translation* MT Summit 2005.   

and
> Shigehiko Schamoni, Felix Hieber, Artem Sokolov, Stefan Riezler. *Learning Translational and Knowledge-based Similarities from Relevance Rankings for Cross-Language Retrieval* ACL 2014.

You can find the PDFs for both papers online with a quick web search.

**Evaluation**: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time (less than 15 minutes on a lab desktop), and you must follow all instructions provided below, including specific implementation requirements. Indexing or translation steps might taken longer than this, however you should upload with your submission preprocessed files (e.g., text files, pickled python structures) to keep the runtime modest.
You will be marked not only on the correctness of your methods, but also on your explanation and analysis. 

Please do not change any of instruction text in the notebook. Where applicable, leave the output cells in the code, particularly when you are commenting on that output. You should add your answers and code by inserting a markdown cell between every major function or other block of code explaining its purpose or anywhere a result needs to be discussed (see the class notebooks for examples). Note that even if you do something wrong, you might get partial credit if you explain it enough that we can follow your reasoning, whereas a fully correct assignment with no text commentary will not receive a passing score. You will not be marked directly on the performance of your final classifier, but each of the steps you take to build it should be reasonable and well-defended.

**Updates**: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

**Academic Misconduct**: For most people, collaboration will form a natural part of the undertaking of this project, and we encourage you to discuss it in general terms with other students. However, it is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s [Academic Misconduct policy](http://academichonesty.unimelb.edu.au/policy.html) where inappropriate levels of collusion or plagiarism are deemed to have taken place.


### Warning: File encodings and tokenisation

All the text files are encoded in *utf-8*, which requires some care in using with python strings, the nltk tools and jupyter.  Please use the following method to convert strings into ASCII by escaping the special symbols. Be careful to do the conversion after tokenisation, as the escaped tags might interfere with the NLTK tokenizer and get treated as punctuation.

In [16]:
from nltk.tokenize import word_tokenize

def tokenize(line, tokenizer=word_tokenize):
    utf_line = line.decode('utf-8').lower()
    return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]

tokenize("Seit damals ist er auf über 10.000 Punkte gestiegen.")

['seit',
 'damals',
 'ist',
 'er',
 'auf',
 '\\xfcber',
 '10.000',
 'punkte',
 'gestiegen',
 '.']

# Part 1: CLIR engine

The project has two parts: first you must build a CLIR engine, comprising information retrieval and translation components, and evaluate its accuracy. Next you will propose an extension to a component of the system. In each step you will need to justify your modelling and implementation decisions, and evaluate the quality of the outputs of the system, and at the end you will need to reflect on the overall outcomes.

The CLIR system will involve:
 - translating queries from German into English, the language of our text collection, using word-based translation
 - once the queries and the documents are in the same language, search over the document collection using BM25
 - evaluate the quality of ranked retreival results using the query relevance judgements

Note that you could try translating the text collection into German instead, but for this project we'll stick with translating the query.

Building the CLIR engine is the majority of work in the assignment, and  constitutes 70% of the assignment mark. Note that although there are several steps, they do not necessarily need to be attempted in a linear order. That is, some components are independent of others. If you're struggling with one component and cannot complete it, then you may be able to skip over it and return to it later. Where outputs are needed in a subsequent step you may want to consider ways of side-stepping this need, e.g., by using Google translate output rather than your system output for translating the queries. You should aim to answer all components.

## Information Retreival with BM25 (20% of assignment mark)

You should implement a vector-space retreival system using the BM25 approach. You'll need to:
 - load in the data files and tokenize the input,
 - preprocess the lexicon, e.g., by stemming and stop-word removal, 
 - calculate the TF/IDF representation for all documents in the collection,
 - store an inverted index such that you can efficiently recover the documents matching a query term, and
 - implement querying in the BM25 model
 - test that it runs with some English queries (you'll have to make these up)
 
This should run in a reasonable time over *dev.docs* (up to a few minutes to index), but beyond this does not need to be highly optimised. Feel free to use python dictionaries, sets, lists, numpy/scipy matrices etc where appropriate for best runtime, but you shouldn't need to use specialised data structures such as compression schemes or the like. Feel free to use APIs in NLTK, scikit-learn and other allowed python modules as needed (see list above.) You will probably want to save and load your index to/from disk to save repeated re-indexing every time you load the file.

In [17]:
from nltk.tokenize import word_tokenize
import pickle
import json
import re
import nltk
from nltk.corpus import stopwords

from nltk.translate.phrase_based import phrase_extraction
from collections import defaultdict, Counter
import itertools
from nltk.tokenize import word_tokenize
from nltk.translate import *
from nltk.translate import IBMModel1
from nltk.translate import AlignedSent, Alignment
import math
stemmer = nltk.stem.PorterStemmer()
stop_word_set=set(stopwords.words('english'))


'''
preprocess_file method
input
filename:the name of document to be indexed

output
tdf: a dictionary of term frequency in each document, {term1:{document1:3,document2:2,...},term2:{document2:4,document5:3},...}
df: document frequency dictionary  {term1:5,term2:6,...}
doc_length: document length dictionary  {document1:213, document2:198,...}
original_docs: a dictionary which contains the original documents

The data file is loaded and tokenized, then stop words are removed and words are stemmed. The output variables are
calculated.
'''
def preprocess_file(filename):
    #dtf={}
    #term document frequency
    tdf={}
    #document frequency
    df={}
    # document length
    doc_length={}
    # a dictionary of original documents. This is stored to retrieve the actual query results
    original_docs={}
    with open(filename,'r') as f:
        for line in f:

            processed_words={}
            #tokenize each sentence
            aWordList=tokenize(line)
            len=0
            for i,word in enumerate(aWordList):

                if i !=0:
                    if word not in stop_word_set:
                        stemmed=stemmer.stem(word)
                        len+=1
                        if stemmed not in tdf:
                            tdf[stemmed]={}
                            tdf[stemmed][aWordList[0]]=1
                        else:
                            tdf[stemmed][aWordList[0]]=tdf[stemmed].get(aWordList[0],0)+1


                        processed_words[stemmed]=processed_words.get(stemmed,0)+1
            original_docs[aWordList[0]]=line
            doc_length[aWordList[0]]=len
            for word in processed_words:
                df[word]=df.get(word,0)+1


            #dtf[aWordList[0]]=processed_words

    return tdf,df,doc_length,original_docs



'''
tokenize method(same as provided in the assignment)
input
line: a string to be tokenized
output
a list of words
'''

def tokenize(line, tokenizer=word_tokenize):
    utf_line = line.decode('utf-8').lower()
    return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]



'''
bm25 method
input
tdf: a dictionary of term frequency in each document
df: document frequency dictionary
doc_length: document length dictionary
num_doc: number of the documents
doc_length_ave: the average length of documents
query: a list of words
k1,b,k3: parameters of BM25 model

output
results: a list of documents sorted according to their scores

This method implements the BM25 model and returns a list of sorted documents.
'''
def bm25(tdf,df,doc_length,num_doc,doc_length_ave,query,k1,b,k3):

    qt=dict()
    # stemming the query
    for term in query:
        if term not in stop_word_set:
            stemmed=stemmer.stem(term)
            qt[stemmed]=qt.get(stemmed,0)+1

    # a dictionary to hold the BM25 score for each document
    results=dict()
    for term in qt:
        if term in tdf:
            for doc in tdf[term]:
                ft=df[term]
                fdt=tdf[term][doc]
                ld=doc_length[doc]
                fqt=qt[term]
                results[doc]=results.get(doc,0)+math.log((num_doc-ft+0.5)/(ft+0.5))*(k1+1)*\
                                                float(fdt)/(k1*((1-b)+b*ld/doc_length_ave)+fdt)*\
                                                (k3+1)*fqt/float(k3+fqt)

    # sort the documents according to their score and store the results into a list
    results=sorted(results, key=results.get, reverse=True)

    return results


tdf,df,doc_length,original_docs=preprocess_file('dev.docs')


print 'tdf,df,doc_length,original_docs completed'
num_doc=float(len(doc_length))
doc_length_ave=sum(doc_length.values())/num_doc

'''
list_to_save=[tdf,df,doc_length,original_docs,num_doc,doc_length_ave]

with open('objs.pickle', 'w') as f:
    pickle.dump(list_to_save, f)
    
    
tdf,df,doc_length,original_docs,num_doc,doc_length_ave=[0,0,0,0,0,0]
with open('objs.pickle') as f:
    tdf,df,doc_length,original_docs,num_doc,doc_length_ave = pickle.load(f)
'''
#the first number represent query ID, which is used to calculate query relevance
# and is useless here
query_example=['123','student','protest','against','too','much','homework']
#default parameters are chosen for the BM25 model
results=bm25(tdf,df,doc_length,num_doc,doc_length_ave,query_example,1.5,0.5,0)

print 'query1 completed', query_example

for docid in results[0:3]:
    print original_docs[docid]
    
query_example2=['123','where','is','the','best','japanese','restaurant']
#default parameters are chosen for the BM25 model
results2=bm25(tdf,df,doc_length,num_doc,doc_length_ave,query_example2,1.5,0.5,0)

print 'query2 completed',query_example2

for docid in results2[0:3]:
    print original_docs[docid]
    
query_example3=['123','school','teacher','student','education']
#default parameters are chosen for the BM25 model
results3=bm25(tdf,df,doc_length,num_doc,doc_length_ave,query_example3,1.5,0.5,0)

print 'query3 completed',query_example3

for docid in results3[0:3]:
    print original_docs[docid]

tdf,df,doc_length,original_docs completed
query1 completed ['123', 'student', 'protest', 'against', 'too', 'much', 'homework']
26194672	postcards from the wedge postcards from the wedge is the fourteenth episode of the simpsons twenty first season it originally aired on the fox network in the united states on march 14 2010 in the episode homer and marge once again try to discipline bart after mrs. krabappel tells them that bart has not been doing his homework but bart has a plan to manipulate homer 's strictness and marge 's sympathetic ear which backfires when homer and marge see through the plan and decide to ignore bart these themes had been seeded in the previous season in episodes like double double boy in trouble and the good the sad and the drugly and would culminate in the show 's first ever true grounding and the first to stand for the rest of the episode the episode was written by brian kelley and directed by mark kirkland the episode features references to the shows pokémon 

<b> Analysis </b>

In the above code, we chose three queries and printed the top 3 results for each query.
The first 2 queries are both sentences, while the third query is a bag of words. Because the BM25 model does not care about the order of the word in the query, the model is more suitable to find documents which is related to a certain area, not answering questions. For example, in the third query, where all the words are related to education, the model did a pretty good job at finding the relevant documents. However, in the first two queryies, because BM25 treat the query as a bag of words, we could not find what we are looking for. Suppose we have two documents, both containing all the words from query 2--one contains the words in exactly the same order as query 2, where we are likely to find the correct answer for "where is the best Japanese restaurant", the other contains the words in arbitrary order, where it is highly impossible to find the answer for our query. Under BM25 model, both documents would be ranked with the same score. That is to say, BM25 does not consider the order of terms in the query, it only cares about the presence and frequency of the words, which is a major drawback.

In addition, in the second query, "where is best Japanese restaurant", the word "where" is probably a high frequency word in the document collection, therefore, under the BM25 model, its impact on the final result becomes less. However, if we analyze the query by its meaning, this query actually intends to find the place of the best Japanese restaurant, not the price of the restaurant, nor the opening hours of this restaurant. BM25 reduces term importance if it appears in many documents, however, in this case, the term "where" carries necessary information for us to solve the query. Although it is useful to reduce the importance of frequent words like "am", "is", "are", "the", "a","what", "where", "which", "when", it is sometimes very dangerous to do so, and this may be another drawback of the BM25 model.


## Translating the queries (40% of assignment mark)

For translation, you should implement a simple word-based translation model in a noisy channel setting, where you search for the best translation string using a language model over the English with a translation model over the back-translation of the English output string, $\vec{e}$, into the German input string, $\vec{f}$. This corresponds to finding the string, $\vec{e}$ which maximises $p(\vec{e}) p(\vec{f} | \vec{e})$. This has two components:

### Language model

The first step is to estimate a language model. You should learn both a unigram language model and a trigram language model, and compare the two. Note that the unigram will be used in the *decoding* step below. You will have to think about how to smooth your estimates, and the related problem of handling unknown words. For smoothing the trigram model, you should you use backoff smoothing. There are a few choices to be made about how much probability mass to assign to the backoff, you will need to decide how to do this -- see the lecture slides and readings for some ideas. Be careful to pad the sentence with sentinel symbols to denote the start and end of each sentence such that the model can predict the words at the start of a sentence, and those when a sentence is complete.

Please use *bitext-large.en* to train your models (start with *bitext-small.en* for your development) and evaluate the perplexity of your model on *newstest2013.en*. You should justify your development decisions and parameter settings in a markdown cell, and quantify their effect on perplexity, including the differences you notice between the unigram and the trigram models.

In [18]:

'''
check_for_unk_train method takes a word and a dictionary of unigram counts as input, and returns
UNK if the word is not in the unigram_counts dictionary
'''
def check_for_unk_train(word,unigram_counts):
    if word in unigram_counts:
        return word
    else:
        unigram_counts[word] = 0
        return "UNK"


'''
convert_sentence_train method takes a sentence(string) and unigram_counts(dictionary) as input,
convert the sentence to the format which can be used to calculate unigram,bigram and trigram
'''
def convert_sentence_train(sentence,unigram_counts):
    return ['<s0>',"<s>"] + [check_for_unk_train(token.lower(),unigram_counts) for token in sentence] + ["</s>",'</s0>']


'''
get_counts method takes sentences( a list of lists, where each list represents a sentence) as input,
calculate unigram counts, bigram counts, trigram counts, token counts, store the results inside 3
dictionaries and a float and return them.
'''
def get_counts(sentences):
    unigram_counts = {}
    bigram_counts = defaultdict(dict)
    trigram_counts=defaultdict(lambda : defaultdict(dict))
    for sentence in sentences:
        sentence = convert_sentence_train(sentence, unigram_counts)
        i=0
        for i in range(len(sentence) - 2):
            trigram_counts[sentence[i]][sentence[i+1]][sentence[i+2]]=\
                trigram_counts[sentence[i]][sentence[i+1]].get(sentence[i+2],0)+1
            bigram_counts[sentence[i]][sentence[i+1]] = bigram_counts[sentence[i]].get(sentence[i+1],0) + 1
            unigram_counts[sentence[i]] = unigram_counts.get(sentence[i],0) + 1

        i+=1
        bigram_counts[sentence[i]][sentence[i+1]] = bigram_counts[sentence[i]].get(sentence[i+1],0) + 1
        unigram_counts[sentence[i]] = unigram_counts.get(sentence[i],0) + 1
    token_count = float(sum(unigram_counts.values()))
    unigram_counts["</s0>"] = unigram_counts["<s0>"]
    return unigram_counts, bigram_counts,trigram_counts, token_count

'''
split_file method takes filename(string) as input, split the file into a list of lists,
where each small list represents a sentence.
'''
def split_file(filename):
    sentences=[]
    with open(filename,'r') as f:
        for line in f:
            sentence=tokenize(line)
            sentences.append(sentence)
    return sentences


'''
check_for_unk_test method checks if the word in the test data is unknown.
'''

def check_for_unk_test(word,unigram_counts):
    if word in unigram_counts and unigram_counts[word] > 0:
        return word
    else:
        return "UNK"

'''
convert_sentence_test method converts test data to the format we need.
'''
def convert_sentence_test(sentence,unigram_counts):
    return ['<s0>',"<s>"] + [check_for_unk_test(word.lower(),unigram_counts) for word in sentence] + ["</s>",'</s0>']


'''
get_unigram_prob_addk
input
sentence: a list of strings
unigram_counts: a dictionary of unigram counts
token_count: the total number of tokens
k: the parameter of unigram perplexity calculation
vocab_count: vocabulary count

output:
the probability of a single sentence

This method calculates the probability of a single sentence using add k smoothing
'''
def get_unigram_prob_addk(sentence, unigram_counts,token_count, k,vocab_count):
    prob_sum=0
    sentence=convert_sentence_test(sentence,unigram_counts)
    for word in sentence:
        prob_sum+=math.log((unigram_counts[word]+k)/ \
                           (token_count+k*vocab_count))
    return prob_sum

'''
get_trigram_prob_backoff
input
sentence: a list of strings
unigram_counts: a dictionary of unigram counts
bigram_counts: a dictionary of bigram counts ( a dictionary of dictionaries)
trigram_counts: a dictionary of trigram counts( a dictionary of dictionaries of dictionaries)
token_count: float
tri_lamda: the parameter of backoff smoothing

output
the probability of a sentence using trigram model and backoff smoothing

'''
def get_trigram_prob_backoff(sentence,unigram_counts,bigram_counts,trigram_counts, token_count,tri_lamda):
    prob_sum=0
    sentence=convert_sentence_test(sentence,unigram_counts)
    leng=len(sentence)
    for i in range(0,leng-2):
        if trigram_counts[sentence[i]][sentence[i+1]].get(sentence[i+2],0)!=0:
            prob_sum+=math.log(trigram_counts[sentence[i]][sentence[i+1]][sentence[i+2]]/ \
                               float(bigram_counts[sentence[i]][sentence[i+1]]))
        elif bigram_counts[sentence[i+1]].get(sentence[i+2],0)!=0:
            prob_sum+=math.log(bigram_counts[sentence[i+1]][sentence[i+2]]/ \
                               float(unigram_counts[sentence[i+1]]))

        else:
            prob_sum+=math.log(unigram_counts[sentence[i+2]]/token_count)

    return prob_sum



'''
calculate_perplexity_unigram method calculates the perplexity of documents using unigram and add k smoothing
input
sentences: a list of lists
unigram_counts: a dictionary of unigram counts
token_count: a floating point number representing the total number of tokens
k: the parameter of add k smoothing

output
perplexity of documents under unigram model and add k smoothing
'''

def calculate_perplexity_unigram(sentences, unigram_counts, token_count, k):
    total_log_prob = 0
    test_token_count = 0
    vocab_count=len(unigram_counts)
    for sentence in sentences:
        test_token_count += len(sentence)+4  # have to consider the start and the end token
        total_log_prob += get_unigram_prob_addk(sentence, unigram_counts,token_count, k,vocab_count)
    return math.exp(-total_log_prob/test_token_count)

'''
calculate_perplexity_trigram method calculates the perplexity of documents using trigram and backoff smoothing
input
sentences: a list of lists
unigram_counts: a dictionary of unigram counts
bigram_counts: a dictionary of bigram counts ( a dictionary of dictionaries)
trigram_counts: a dictionary of trigram counts( a dictionary of dictionaries of dictionaries)
token_count: a floating point number representing the total number of tokens
tri_lamda: the parameter of backoff smoothing

output
perplexity of documents under trigram model and back off smoothing
'''
def calculate_perplexity_trigram(sentences, unigram_counts,bigram_counts,trigram_counts, token_count,tri_lamda):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 2 # have to consider the end token
        total_log_prob += get_trigram_prob_backoff(sentence,unigram_counts,bigram_counts,trigram_counts, token_count,tri_lamda)
    return math.exp(-total_log_prob/test_token_count)


train_sentences=split_file('bitext-large.en')

unigram_counts, bigram_counts,trigram_counts, token_count=get_counts(train_sentences)

print 'unigram_counts, bigram_counts,trigram_counts, token_count completed'


test_sentences = split_file('newstest2013.en')
print "unigram perplexity,using add k, k= 1,2,3"
print calculate_perplexity_unigram(test_sentences,unigram_counts,token_count,1)
print calculate_perplexity_unigram(test_sentences,unigram_counts,token_count,2)
print calculate_perplexity_unigram(test_sentences,unigram_counts,token_count,3)
print "trigram perplexity"
print calculate_perplexity_trigram(test_sentences,unigram_counts,bigram_counts,trigram_counts,token_count,1)

unigram_counts, bigram_counts,trigram_counts, token_count completed
unigram perplexity,using add k, k= 1,2,3
611.82796244
614.95546767
618.772985829
trigram perplexity
132.539743873


<b> Analysis</b>
For unknown words, during the training phase(extract unigrams, bigrams, trigrams), whenever we come across a new word, we assume it is an unknown word, and substitude it with "UNK". During the testing phase, for words that do not appear in unigram_counts or the value of unigram_counts for the words is 0, we substitude the words with "UNK". 

Since unigram_counts, bigram_counts, and trigram_counts are calculated inside the same function, the sentences are all padded with "< s 0 >""< s >" "< / s >""< / s 0 >", which is suitable for trigram calculation. 

For the perplexity calculation of unigram language model, we used add k smoothing, and print the perplexity results of k=1,k=2,k=3. As k gets larger, the final perplexity gets larger, and the likelihood of test data gets smaller. This reason for this may be illustrated by an example. Assume there are 5 different words in total in the training dataset, each appearing only once, and 4 different words in total in the test dataset, also with one appearance each. The probability of each word during the training phase is 1/5. If we calculate perplexity for test dataset using add 2, then it would roughly be  1/(((1+2）/(5+5*2))^4) ( square root is ommited, as it does not affect the trend.) Adding 3 would be 1/(((1+3）/(5+5*3))^4), as k gets larger, this number would get larger. Since the size our test dataset is much smaller than the training set, it is reasonable to make the above assumption.

The perplexity of the testing dataset using trigram is much smaller than that using unigram. This agrees with our assumption that the test document makes more sense under the trigram model. Because when calculating the perplexity using trigram, the probability of each word is calculated based on its previous words, for example, if the previous words are "she goes", then the next word is highly likely to be "to". In similar manner, the probability of each word increases, and the likelihood of whole text gets larger, resulting in the perplexity getting smaller.


### Translation model

The next step is to estimate translation model probabilities. For this we will use a word-based alignment model, e.g., *ibm1* to learn word-based translation probabilities using expectation maxisation. Your task is to obtain good quality word alignments from this data, which will require careful use of the word alignment models. You will want to training in both translation directions, and combining the alignments.

You should use the *bitext-small* files for this purpose, and be aware that it may take a minute or two to train ibm1 on this data. Inspect some of the word alignments and translation probabilities (e.g., using a few common words in German such as *haus*) to see if your approach is working, and give some examples of output alignments that you find that are good and bad. 

In [19]:

'''
read_bitext methods reads two files and returns a list of aligned sentences
'''
def read_bitext(file_name1,file_name2):
    bitext=[]
    with open(file_name1,'r') as f1:
        with open(file_name2,'r') as f2:
            for line1,line2 in zip(f1,f2):
                list1=tokenize(line1)
                list2=tokenize(line2)
                bitext.append(AlignedSent(list1,list2))
    return bitext

'''
Transform the translation table of English to German, for easier processing in the next steps
'''

def n_translation_table(t_en_de):
    n_t_en_de=defaultdict(dict)
    for en in t_en_de:
        for de in t_en_de[en]:
            if de in n_t_en_de:
                n_t_en_de[de][en]=n_t_en_de[de].get(en,0)+t_en_de[en][de]
            else:
                n_t_en_de[de][en]=t_en_de[en][de]
    return n_t_en_de




'''
combine_translation_table takes two translation tables(from en to de and from de to en),
unigram_counts and token_count as input, combine the two tables together
and return the combined table. The table are combined by adding p(e|f) and p(f|e)p(e)
together
'''
def combine_translation_table(t1,t2):
    t=defaultdict(dict)
    
    for word1 in t1:
        if word1 in t2:
            for word2 in t1[word1]:
                t[word1][word2]=t1[word1][word2]+\
                t2[word1].get(word2,0)
        else:
            for word2 in t1[word1]:
                t[word1][word2]=t1[word1][word2]
    return t

bitext1=read_bitext('bitext-small.de','bitext-small.en')

ibm1 = IBMModel1(bitext1, 5)
t1=ibm1.translation_table


print 'translation table 1'
print 'haus,house',t1['haus']['house']
print 'unser, our',t1['unser']['our']
print 'im,in',t1['im']['in']
print 'gut,good',t1['gut']['good']

print 'alignment 1'
for i in range(0,5):
    print `bitext1[i]`


bitext2=read_bitext('bitext-small.en','bitext-small.de')

ibm1reversed = IBMModel1(bitext2, 5)
t2=ibm1reversed.translation_table


print 'translation table 2'
print 'haus,house',t2['house']['haus']
print 'unser, our',t2['our']['unser']
print 'im,in',t2['in']['im']
print 'gut,good',t2['good']['gut']

print 'alignment 2'
for i in range(0,5):
    print `bitext2[i]`

n_t2=n_translation_table(t2)
t_combined=combine_translation_table(t1,n_t2)

print 'combined translation table'
print 'haus,house',t_combined['haus']['house']
print 'unser, our',t_combined['unser']['our']
print 'im,in',t_combined['im']['in']
print 'gut,good',t_combined['gut']['good']

translation table 1
haus,house 0.414602078104
unser, our 0.208992664598
im,in 0.134873601334
gut,good 0.308055968157
alignment 1
AlignedSent(['steigt', 'gold', 'auf', '10.000', 'dollar', '?'], ['$', '10,000', 'gold', '?'], Alignment([(0, 1), (1, 2), (2, 1), (3, 1), (4, 0), (5, 3)]))
AlignedSent(['san', 'francisco', '\u2013', 'es', 'war', 'noch', 'nie', 'leicht', ',', 'ein', 'rationales', 'gespr\xe4ch', '\xfcber', 'den', 'wert', 'von', 'gold', 'zu', 'f\xfchren', '.'], ['san', 'francisco', '\u2013', 'it', 'has', 'never', 'been', 'easy', 'to', 'have', 'a', 'rational', 'conversation', 'about', 'the', 'value', 'of', 'gold', '.'], Alignment([(0, 0), (1, 1), (2, 2), (3, 3), (4, 6), (5, 1), (6, 5), (7, 7), (8, 8), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 8), (18, 11), (19, 18)]))
AlignedSent(['und', 'es', 'kam', ',', 'wie', 'es', 'kommen', 'musste', '.'], ['wouldn\u2019t', 'you', 'know', 'it', '?'], Alignment([(0, None), (1, 3), (2, 0), (3, 1), (4, 0)

<b> Analysis </b>

In the above code, we firstly read the bitext in and store it in the format required by the IBM models. Then IBM model1 is trained and a translation table is ourputed. To get a shallow understanding of the quality of the translation table, we printed the translation probablity for some common words, namely "haus" and "house", "unser" and "our", "im" and "in", "gut" and "good". Form the output, we can see the probability of these word pairs are relatively high, meaning that the model realizes they are corresponding pairs in the two languages. 

For the sentence alignments, the first 5 alignments are printed. As we can see from the results, the overall alignment is not bad. For the first sentence, it successfully aligned the dollar sign with dollar, gold with gold. For the second sentence, it successfully aligned it with es, rationales with rational, wert with value and so on. When the model is trained backwards, the results are similar. 

To combine the two translation tables together, we are simply summing p(e|f) and p(f|e) together, where p(e|f) according our code is translation table t1, p(f|e) is translation table t2. To illustrate this, I'll use an example.  t1 represents the probability of haus translated to house, t2 represents the probability of house translating to haus, since the models for these two tables are trained in different order, there may be some difference between them, by summing them together, we manage to minimize the errors of a single model. Although the results of this would be in the range of 0~2, since we only care about the relative probabilities compared to other translations, this would be fine. 

From the combined probabilities, we can see that although the probability of 'unser' being translated into 'our' is low in translation table 1, now after combining the probability from table 2, the probability become higher. And the same rule applies to all the other words. This combination minimize the errors of a single model.


### Decoding 

Finally you'll need to solve for the best translation for a given string. For this you should do simple word-for-word translation where each word is translated independently. Use the alignments learned above (or the translation parameters) to translate each word of the queries into English. Compare taking the maximum probability of translation into English $p(e|f)$, with the noisy-channel probability, $p(f|e)p(e)$, which considers the reverse translation and a unigram language model probability. (You'll get a chance later to do something more ambitious, using the trigram model.)

Use your algorithms to translate the first 100 queries into English, and save them to disk.

In [20]:
'''
decoder_de_to_en method takes t_de_en( a translation table from de to en) and q_list(a list of sentences to be
translated) as input, translats the sentences into en by choosing the word with maximum p(e|f). If the word in
the query does not exist in the translation table, it is translated into an empty string.
'''

def decoder_de_to_en(t_de_en, q_list):
    translated=[]
    for sent in q_list:
        #print sent
        tran_sent=[sent[0]]
        for de_word in sent[1:]:
            best_tran=''
            if de_word in t_de_en:
                best_prob=0
                for en_word in t_de_en[de_word]:
                    if t_de_en[de_word][en_word]>best_prob:
                        best_prob=t_de_en[de_word][en_word]
                        best_tran=en_word
            tran_sent.append(best_tran)
        #print tran_sent
        translated.append(tran_sent)
    return translated

'''
decoder_en_en_to_de method takes t_en_de( a translation table from en to de), q_list( a list of sentences to
be translated), and unigram_counts as input, translates the q_list into en by choosing the word with maximum
P(f|e)p(e). If the word in the query does not exist in the translation table, it is translated into an empty
string.
'''

def decoder_en_en_to_de(n_t_en_de,q_list,unigram_counts):
    
    translated=[]
    for sent in q_list:
        tran_sent=[sent[0]]
        for de_word in sent[1:]:
            best_tran=''
            if de_word in n_t_en_de:
                best_prob=0
                for en_word in n_t_en_de:
                    # we are dividing unigram_counts by token counts because we only care about
                    #the relative amount
                    prob= unigram_counts.get(en_word,0)*n_t_en_de[de_word].get(en_word,0)
                    if prob>=best_prob:
                        best_prob=prob
                        best_tran=en_word
            tran_sent.append(best_tran)
        translated.append(tran_sent)

    return translated

quries=split_file('dev.queries')

# translated queries 1 is calculated by choosing the largest p(e|f)
trans_quries1=decoder_de_to_en(t1,quries[0:100])
# translated queries 2 is calculated by choosing the largest p(f|e)p(e)
trans_quries2=decoder_en_en_to_de(n_t2,quries[0:100],unigram_counts)
# translated queries 3 is calculated by choosing the largest p(e|f) using the combined table
trans_quries3=decoder_de_to_en(t_combined,quries[0:100])

print 'translated queries 1, using maximum p(e|f)'
for i in range(0,5):
    print trans_quries1[i]
print 'unigram perplexity'
print calculate_perplexity_unigram(trans_quries1,unigram_counts,token_count,1)
print 'trigram perplexity'
print calculate_perplexity_trigram(trans_quries1,unigram_counts,bigram_counts,trigram_counts,token_count,1)
    
print 'translated queries 2,using maximum p(f|e)p(e)'
for i in range(0,5):
    print trans_quries2[i]
print 'unigram perplexity'
print calculate_perplexity_unigram(trans_quries2,unigram_counts,token_count,1)
print 'trigram perplexity'
print calculate_perplexity_trigram(trans_quries2,unigram_counts,bigram_counts,trigram_counts,token_count,1)

    
print 'translated queries 3,using combine translation table'
for i in range(0,5):
    print trans_quries3[i]
print 'unigram perplexity'
print calculate_perplexity_unigram(trans_quries3,unigram_counts,token_count,1)
print 'trigram perplexity'
print calculate_perplexity_trigram(trans_quries3,unigram_counts,bigram_counts,trigram_counts,token_count,1)

translated queries 1, using maximum p(e|f)
['82', 'deterring', '(', 'von', 'guises', '.', 'affirmative', ':', 'subset', 'which', 'abhor', 'which', 'movement', ')', 'is', 'arrive', '', 'gambler\\u2019s', '', 'which', 'into', '\\u201cabenomics\\u201d', 'deterring', 'neuro-scientist', 'deterring', 'external', 'abhor', 'von', '5-7', '', 'remnants', 'struggle', '-', 'and', '', 'embryonically', 'and', 'states-china', 'becomes', '.']
['116', 'chronicles', '(', '', ':', '', 'for', '', 'bulletin', 'cambridge', '', 'which', 'outdated', '', 'for', 'bulletin', 'cambridge', '', ')', 'is', 'embarked', 'befuddled', 'deterring', 'en', '.']
['240', 'deterring', 'von', '', '', 'which', '``', 'really', '``', 'which', 'also', '', 'which', '', '-', 'or', '', 'which', 'english', '', 'which', 'is', 'chronicles', 'fundamental', 'scientific', 'method', 'into', 'deterring', '.']
['320', 'chronicles', '(', 'faros', 'el', 'which', 'von', '', '', 'which', '-', '``', 'together', '-', '``', 'which', '``', '', '``', 

Now you will need to write some text about the procedure above, and how well it worked. You should answer:
  - how good are the translations? How do your methods fare at translating the content words versus functions words? You can use Google translate to provide an alternative (pretty good) translation for comparison purposes. 
  - what are good settings for various modelling parameters such as language model discounting, translation model smoothing, any decoding parameters, and how do these affect the outputs?
  - compare the language model perplexity for the unigram and trigram models over your decoder outputs, how does the difference in perplexity between the two models compare to your test above on the monolingual data? 
  - how do the learned alignments differ between the two translation directions, and does their combination appear oto improve the predictions? (You don't need a formal evaluation here, as we have no *gold standard* alignments.)

<b> Analysis </b>

In the above code, we used 3 methods to translate the queries. The first method simply chooses the translation with largest p(e|f), using the translation table of German to English. The second method chooses the translation with largest p(f|e)p(e), using translation table of English to German, and unigram_counts dictionary. The third method uses the combined translation table(the table is combined by adding p(e|f) and p(f|e) together), and chooses the translation with largest probability. 
A brief glipse of the results shows that the second method is of lowest quality, and is more suitable for translating function words than content words, since the output we get actually consists of only function words, which would be of no use for querying corpus. Actually all the methods are better at translating the function words, this is because function words are much more common during training, and our model has good knowledge of these words.

To evaluate the quality of translation method 1 and 3, the results of google translation of the same queries are shown here:
82 (. Of eng action: act, action, movement,) is a film genre of entertainment cinema, in which the continued transition of the external action of mostly spectacularly staged battle - and violent scenes is advanced and illustrated.

116 (unit sign: u for unified atomic mass unit, amu outdated for atomic mass unit) is a unit of measure of mass.

240 of actualis from Latin, "real", even actuality principle, uniformity - or gleichförmigkeitsprinzip, English uniformitarianism, is the basic scientific method in.

320 (Greek el, from Ancient Greek grc, - "together -", "tie", is meant "the heart bag attached") is a blood vessel that leads away the blood from the heart.

540 under the designation one summarizes the three lying in the northern waters alpenvorland units obersee, subsea and Seerhein together.

After carefully comparison, we found out the performance method 3 is slightly better than that of method 1. For example, the translation using method 3 translates "action", "mass", "from" and so on correctly. Nonetheless, the performance of both models are actually poor. This is because we are translating only by words, and the training dataset is too small that many words we meet during translation are not contained in translation tables.

The perplexity of method 2 is the smallest, since a lot of words did not get translated in method 2 and is replaced with an empty string(which will converted to "UNK" during calculation), and the only words get translated are all function words, of which the word probabilities are high, this results in an abnormal perplexity score. 

The unigram perplexity scores of method 1 and 3 are actually lower than that of monolingual data, this is because during translation, the word with higher probability of appearance is chosen, and many words did not get translated. Unigram perplexity does not take any word order into consideration, therefore, a bag of high frequency words has smaller unigram perplexity. 

The trigram perplexity scores of method 1 is higher than method 3, and the score of method 3 is similar to that of monolingual data. The good score of method 3 is probably because many word is translated into empty strings, and these empty strings are turned into "UNK" during calculation, and the probability of "UNK" relatively high. In addition, many words do not appear in the trigram models, thus our backoff algorithm will use bigram, then unigram, which will discard the order information gradually. 

The performance of a combination of two translation directions is slightly better by examining the first 5 translated queries, and also by comparing the perplexity score, the reason for this has already been illustrated above. 



## Putting the CLIR system together (10% of assignment mark)

Now you should couple the translation and IR components from above. Take your translated queries and use these with the IR system to find the most relevant documents. You should then evaluate these predictions against the supplied relevance judgements, using the *mean average precision* (MAP) metric. As part of this, you will want to consider tuning the settings of the IR system for best performance, and comment on how successful your approach was and evaluate the IR performance is affected by the parameter settings and modelling decisions you have made.

Note that if you were stuck on the translation steps above, or your translations are otherwise unusable, you can implement this step using google translate outputs for the 100 queries.

In [21]:

'''
read_query_rel method reads in query relevance file and processes it into a dictionary of dictionaries,
where keys are the query id and document ids, and the values are the relevance levels
'''
def read_query_rel(filename):
    query_rel=defaultdict(dict)
    with open(filename,'r') as f:
        for line in f:
            line=line.split('\t')
            query_rel[line[0]][line[2]]=line[3]

    return query_rel


'''
map method calculates mean average precision of the first k query results using query_rel dictionary.
'''
def map(query_results,query_rel,k):

    map1=0
    for query in query_results:
        rel_levels=0
        rel_num=0
        precision_sum=0
        for i,docid in enumerate(query_results[query][0:k]):
            if docid in query_rel[query]:

                rel_num+=1
                #print 'rel_num', rel_num
                precision=rel_num/float(i+1)
                #print 'precision',precision
                precision_sum+=precision

        if rel_num==0:
            ave_precision=0
        else:
            ave_precision=precision_sum/rel_num
            #print 'ave_precision',ave_precision


        #if ave_precision>2:
            #print query,query_results[query][0:k]
        map1+=ave_precision



    map1=map1/float(100)
    return map1

# We tuned some parameters to see if it changes the output map
query_rel=read_query_rel('dev.qrel')
for k1 in [1,1.5,2]:
    for b in [0.5,0.9]:
        for k3 in [0,2]:
            total_result=dict()
            for i,query in enumerate(trans_quries3):
                total_result[query[0]]=bm25(tdf,df,doc_length,num_doc,doc_length_ave,query[1:],k1,b,k3)

            
            map1=map(total_result,query_rel,100)

            print'k1',k1,'b',b,'k3',k3,'map', map1

k1 1 b 0.5 k3 0 map 0.173657625218
k1 1 b 0.5 k3 2 map 0.162724869763
k1 1 b 0.9 k3 0 map 0.17159011618
k1 1 b 0.9 k3 2 map 0.156905172064
k1 1.5 b 0.5 k3 0 map 0.180242378134
k1 1.5 b 0.5 k3 2 map 0.172917969815
k1 1.5 b 0.9 k3 0 map 0.172733374121
k1 1.5 b 0.9 k3 2 map 0.163368747105
k1 2 b 0.5 k3 0 map 0.185343637229
k1 2 b 0.5 k3 2 map 0.176912458099
k1 2 b 0.9 k3 0 map 0.17533658329
k1 2 b 0.9 k3 2 map 0.161841381465


<b> Analysis </b>

As we can see, the mean average precision is always low. We tried 3 values for k1, namely 1,1.5, 2, two values for b, 0.5, 0.9, two values for k3, 0 and 2. The mean average precision is calculated using the first 100 results for each query. As can be seen from the outputs, the results does not change much in terms of mean average precision. This is probably because the translation results for the queries are already poor and almost not related to the original queries, therefore, no matter how the parameters are tuned, it cannot improve the search results.

However, an interesting fact to notice is that the result of k3=0 is always better than that of k3=2. k3 controls the influence of the frequency of terms in the query. k3=0 means the frequency of terms in the query does not influence the final results at all, and as k3 gets larger, the more frequent words play a more important role in the querying. From our result, we can see that k3 is better set as 0.

Note that in a complete CLIR you would translate the retreived documents back into German. We won't bother about that for this assignment, especially as I doubt many of you can understand German.

# Part 2: Extension and discussion

## Extension (25% of assignment mark)

You now have a working end-to-end CLIR system. The next step is to revisit some of the steps above to see if you can improve the performance of the system. You are invited to be creative with your ideas about how to do this. Note that many great ideas might not produce improvements, or end up being very difficult to implement. This will not be assessed solely on the grounds of CLIR accuracy; we will be looking primarily for interesting and creative ideas.

Here are some ideas on what you might do. Warning: some are bigger than others, aim to spend 5-6 hours in total on this.

#### Structured decoding

Decode using a higher order language model, possibly with word reordering. Using your trigram language model and a word-based or phrase-based translation model, search for the best scoring translation sentence. You may want to use the tools in *nltk.translate*, including the phrase extraction and *StackDecoder*, especially if you wish to support word reordering. The decoding algorithm is much simpler (and tractable) without reordering, so you might want to implement this yourself. You will want to work with log-probabilities to avoid numerical underflow. Note  that the *StackDecoder* is not highly stable code, so you will need to understand its code in detail if you chose to use it. You will need to supply translation log probabilities and language model log probabilities as input to the decoder and consider the context of the previous two words in estimating the probability of each word. 

#### Probabilisitc querying

Consider queries including weighted terms based on multiple translation outputs, such as the word translation probabilities or using *k-best* translations from a decoder (you will need to extract not just the best translation from the decode, but the runners up, which can be done from the stack contents after beam search.)  You should consider how best to define TF and DF for query words based on their ambiguous translations. You may find inspiration in the bibliography of [Douglas Oard](https://terpconnect.umd.edu/~oard/research.html#mlia) such as [this paper](https://terpconnect.umd.edu/~oard/pdf/coling12ture.pdf).

#### Word alignment

Implement an extended word alignment model, such as the *hmm* model (see [Vogel's paper](http://www.aclweb.org/anthology/C96-2141)) or a variant of *ibm2*. The variant of *ibm2* could include a better formulation for the distortion term, $p(i|j,I,J)$, based around rewarding values of i and j that are at similar relative positions to encourage alignments near the diagonal. This contrasts with *ibm2* which just learns a massive table of counts for all combinations of i,j,I and J. This strictly needs normalisation, as its not a probability, but for simplicity you can ignore this aspect here. This idea is inspired by [fast-align](https://github.com/clab/fast_align) which has a similar, but slightly more complex, formulation.

You might also want to experiment with [fast-align](https://github.com/clab/fast_align) (an extremely fast and accurate variant of *ibm2*) or [GIZA++](http://www.statmt.org/moses/giza/GIZA++.html) (an implementation of *ibm1* - *ibm5* and the *hmm*), both widely used and highly optimised alignment tools. Neither of these tools are in python, and may be a little involved to install and compile.

#### Decoding

Experiment with a state-of-the-art translation tool like [Moses](http://www.statmt.org/moses/).  Can you get better CLIR performance using this, trained on the full parallel dataset? You might want to consider translating the document collection into German, and then doing the IR entirely in German. You might consider building a python interface to moses, building on the existing very basic funcationality. **Warning:** Moses is a complex suite of software, and can be a little complex to install and use. However there are good installation and usage guides on the moses webpage.

#### IR engine

There are several great open source IR engines, including [Lucene](https://lucene.apache.org/core/), [Terrier](http://terrier.org/), [Zettair](http://www.seg.rmit.edu.au/zettair/) and [Lemur](http://www.lemurproject.org/). You could experiment with these, and use this to compare the accuracy of different types of IR models on the CLIR data. Alternatively, you could implement a faster or more compact retreival system in python, e.g., using compressed vbyte encodings, skip lists or similar.

Note that the development of some of the extensions maye require working in the command shell and in other languages than python. If this is the case, you should attach your code in separate files as part of your submission, but include the analysis text here. *Note that excellent submissions implementing interfaces or models not currently in NLTK will be considered for inclusion into the toolkit.* You should include both the code, and text explaining your work and findings.

In [22]:
print "start IBMModel1"
bitext2=read_bitext('bitext-small.en','bitext-small.de')
#train the model so that we get initial alignments
model_ext = IBMModel1(bitext2, 5)

phrase_pair_counts = defaultdict(Counter)
for s in bitext2:
    #print `s`
    flag=False
    # some of the alignments contains none, which will make the phrase extraction throw exceptions
    # we simply discard them
    for i,align in enumerate(s.alignment):
        for num in align:
            if num==None:
                flag=True
    #print `s`
    if not flag:
        phrases = phrase_extraction(' '.join(s.words), ' '.join(s.mots), s.alignment, max_phrase_length=3)
        #print phrases
        for en_span, fr_span, en_phrase, fr_phrase in phrases:
                for en_span, fr_span, en_phrase, fr_phrase in phrases:
                    phrase_pair_counts[tuple(en_phrase.split())][tuple(fr_phrase.split())] += 1
#Calculate a phrase table
phrase_table = PhraseTable()
for en_phrase, fr_counts in phrase_pair_counts.items():
    total = sum(fr_counts.values())
    for fr_phrase, count in fr_counts.items():
        phrase_table.add(fr_phrase, en_phrase, math.log(count / float(total)))


# calculate a language probability model
language_prob = defaultdict(lambda: -999.0)
total_words = sum(unigram_counts.values())
for word_type, count in unigram_counts.items():
    if count>0:
        language_prob[word_type] = math.log(count / float(total_words))

# I tried to implement a decoder myself, which will take phrase table, and trigram_counts as 
#input, and translate the sentence based on phrase translation, but it turns out that there are 
#so many conditions to consider and the process is too complicated, the following is an implementation
#that is not yet finished
def my_decoder(phrase_table, trigram_counts, bigram_counts, unigram_counts, query):
    i=0
    best_tran_query=[]
    while i < (len(query)-2):
        best_tran=[]
        best_prob=-9e99
        try:
            trans=phrase_table.translations_for(tuple(query[i:i + 3]))

            for a_trans in trans:
                prob_trans=a_trans[1]
                trans_words=list(a_trans[0])
                if len(trans_words)==3:
                    if trigram_counts[trans_words[0]][trans_words[1]].get(trans_words[2], 0)!=0:
                        prob_lan=math.log(trigram_counts[trans_words[0]][trans_words[1]][trans_words[2]] / \
                                          float(bigram_counts[trans_words[0]][trans_words[1]]))

                else:
                    prob_lan=-9e99

                prob=prob_lan+prob_trans
                if prob>=best_prob:
                    best_tran=trans_words


            i+=3

        except KeyError:
            i+=1


        best_tran_query.append(best_tran)
    return best_tran_query


# wrapper class
class LM:
    def __init__(self, probs):
        self.probs = probs
    def probability_change(self, context, phrase):
        # Used when expanding a hypothesis with a new phrase
        # (higher order LMs would need to look at the word sequence, including context)
        return sum([self.probs[word] for word in phrase])
    def probability(self, phrase):
        # Used for future cost estimation only, to get a cheap (approximate) LM score for each phrase
        return sum([self.probs[word] for word in phrase])        
        
    

language_model = LM(language_prob)
# this stack_decoder fails when the words in the queries do not appear in the phrase table or
#language model
stack_decoder = StackDecoder(phrase_table, language_model)
# it only works with some common words
print stack_decoder.translate(['unser','im'])
# our not yet finished decoder
for query in quries[0:5]:

    print my_decoder(phrase_table, trigram_counts, bigram_counts, unigram_counts, query[1:])

start IBMModel1
['our', 'in']
[[], [], [], [], [], [], [], [], [], [], [], [], [')', 'is', 'a'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [')', 'is'], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['the'], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [')', ','], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


<b> Analysis </b>

For this part, I tried the first option "Structured decoding". The stack decoder from nltk does not work when the words from queries do not exist in the phrase table or language model. Therefore I chose to implement a decoder myself. The initial alignments is calculated using IBM model 1, then these alignments used to calculate a phrase table. The two inputs to the decoder should be phrase table and language model. Since we intend to translate by phrase, the language model should be a table of trigram language probabilities. However, our training data and computing resource is very limited, which means the trigram model can not cover all the phrases in the queries. Ideally, when the phrase does not exist in trigram, we should turn to bigram, then unigram. The probability of a word is calculated by multiplying its phrase table probability and trigram probability giving its previous two words. And the translation process is to choose the word which gives the largest probability. 

Unfortunately, the implementation of this decoding algorithm turns out to be too complicated. We can represent this by pseudo code(in ideal case, when the 3-word-long phrase exists in phrase table and trigram model)


for a phrase(3 words long) in queries:

    if phrase(3 words long) in phrase table:
        for all the phrase translations in phrase table:
            if phrase translation in trigram model:
                probability of translation =phrase table probability*trigram probability


then the phrase translation with largest probability product is chosen. 

However, it is possible that the phrase does not exist in phrase table, for this case, we have to first shorten the phrase to 2 words first, then check if it exist in phrase table
     
     else:
         if phrase(2 words long) in phrase table:
             for all the phrase translations in phrase table:
                 if phrase translation in bigram model:
                     probability of translation =phrase table probability*bigram probability

It is also possible that even the 2-word-long phrase does not exist in phrase table, in this case, the word is translated simply using the previous method(choosing the largest p(e|f)
    
In the above algorithm, we assumed that if the phrase exists in the phrase table, it exists in the trigram or bigram model. It is also possible that the phrase does not exist in trigram or bigram model. 
For the 3-word-long phrase, if it does not exist in trigram model, we will use the product of two bigram probabilities to represent the probability of the 3 words. For the 2-word-long phrase, if it does not exist in bigram, we will use two unigram probabilities. 

The above algorithm is pretty much my understanding of translation by phrase. As it contains too many "if"s, when I tried to implement it, it becomes too messy, so I only wrote down my thoughts here. 



## Discussion (5% of marks)

What conclusions can you make about CLIR, or more generally translation and IR? Overall what approaches worked and what ones didn't? What insights do you have from doing this, put another way, if you were to start again, what approach would you take and why?

```
There are several conclusions I learned from this assignment.

Firstly, the capacity of training data and the computing resource available is crucial for machine translation. Our first translation model is built based on translation by word. However, even we are only considering single words, there are still many words in the queries that did not appear in the training data, which means these words are impossible to get translated. On the other hand, if we have more training data, and computing resource allows us to train the model more times, then it is highly likely that we could get better alignments for the existing words and also the size of vocabulary covered would be larger.

Secondly, it is important to take the order of words into consideration during translation. Sometimes one words would correspond to more than one words, and in this case, word by word translation would be meaningless. However, if the sentence is translated by phrases, then although one word in the phrase is not very common and of low probability, the phrase as a whole has high probability, and in this way it is likely to get the whole phrase correct.

Nonetheless, although translating by phrase could be useful, it highly relies on the training data. If the training dataset is too small, many phrases in the test data would not be covered, and there would be no way but use word by word translation.

Thirdly, the BM25 model assumes common words carry less information, which I found is a little questionable. As our results above shown, if the query includes words like where or when, which are quite common words inside the document collection, these words would be given less weights. If the query is "where is the best Japanese restaurant" and one document contains "Japanese restaurants" many times, but the word "where" is missing, this document is likely to be ranked high in the query results, but is unlikely to be the correct result since there is probably no location information.

Finally, BM25 also treats queries as a bag of words, which will obiviously affect the performance. If both documents contain the words in the query, one in the same order as the query, another in arbitrary order, they will be ranked the same using BM25 model. From experience, we know the first one is actually better.

If I were to start again, I would do the translation part using phrase table and trigram language model, given that there is enough data to get a phrase table of good quality. By good quality I mean most of the phrases in the test data would exist in the phrase table. If the training dataset is large enough, it is reasonable to assume that most of phrases in the test set has corresponding entries in the phrase table and trigram language model.

During the querying phase, sometimes there are also phrases in the queries, if we could search for these phrases in the documents as a whole, this would increase the accuracy of the result. We coud segment the queries in various ways and compare the returned results and choose the best ones.

```